# Working with Maxwell-filtered data


`
Author: Britta Westner
`

In this notebook, we will have a look at the impact of Maxwell filtering (also known as Signal Space Seperation or SSS) on beamformer source reconstruction. We will see the impact of Maxwell filtering on data and rank and how to remedy this for beamformer source reconstruction. We use the `sample` dataset, which ships with SSS-files.

The solutions discuss in this notebook can also hold for other severely rank-deficient data.

In [1]:
%matplotlib inline
import mne

mne.set_log_level('warning')

# set paths:
sample_path = mne.datasets.sample.data_path()
data_path = sample_path / 'MEG' / 'sample'
subjects_dir = sample_path / 'subjects'

## Read and Maxwell-filter raw data

First, we read the raw data from disk. We only keep the MEG data, as we are not interested in the EEG data for this tutorial. We also keep the stim channels, because we still want to cut the data in to epochs later. 

We also have to find the SSS files in the `sample` data folders, so we set the paths below.

In [2]:
# read raw data
fname_raw = data_path / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(fname_raw)
raw.pick(picks=['grad', 'mag', 'stim'])

# set the paths to the SSS files
fname_fine_calib = sample_path / 'SSS' / 'sss_cal_mgh.dat'
fname_crosstalk = sample_path / 'SSS' / 'ct_sparse_mgh.fif'


Note that this is not a tutorial on how to properly Maxwell-filter your data. If you are curious about this, you can check out [the Maxwell filtering tutorial](https://mne.tools/stable/auto_tutorials/preprocessing/60_maxwell_filtering_sss.html) on the MNE-Python homepage.

Usually, we would start off with detecting bad channels. In this data set, the channel that would be returned as bad is already marked as bad in our data set anyway: `MEG 2443`. Another one would only be detected by visual inspection, which we will not perform here (check out the tutorial above if you are curious about this step). We will simply mark it as bad manually.

In [3]:
raw.info['bads'] += ['MEG 2313']  # add extra channel from the Maxwell filtering tutorial to bads

You can check if this worked by looking at the information in `raw.info`:

In [ ]:
raw.info

### Maxwell filtering steps

Now, we can move on to clean our data with Maxwell filtering, using the files we have identified above: the cross talk file and the fine calibration.

In [5]:
raw_sss = mne.preprocessing.maxwell_filter(raw,
                                           cross_talk=fname_crosstalk, calibration=fname_fine_calib)

## Compute epochs

Next, we want to create epochs from the continuous data and compute the evoked fields.
We will do this for both the filtered and original data - to see the effect Maxwell filtering has on the signal.
We use the epochs where a visual stimulus was presented in the left or right visual field. These have the trigger codes `3` and `4`.

From the epochs, we create the evoked fields.

In [6]:
events = mne.find_events(raw)   # the stim channels stay the same, so we can use the same events object
epochs_orig = mne.Epochs(raw, events, event_id=[3, 4])
evoked_orig = epochs_orig.average()

epochs_sss = mne.Epochs(raw_sss, events, event_id=[3, 4])
evoked_sss = epochs_sss.average()

Let's plot the evoked fields. You can see that there are some differences between the original and the cleaned data. However, for this dataset, the differences are not too big. This of course can be very different for other recording sites - or different experiments (e.g., if you have equipment present that might introduce noise).

In [ ]:
evoked_orig.plot(titles=dict(mag='Original data, magnetometers', grad='Original data, gradiometers'));
evoked_sss.plot(titles=dict(mag='Maxwell filtered data, magnetometers', grad='Maxwell filtered data, gradiometers'));

## The impact of SSS on data rank

Let's have a look, however, what else Maxwell filtering did to our data. Maxwell filtering removes noise components from the data. This removal means that the data becomes rank-deficient. 

Let's have a closer look at what this means: the **rank** of a matrix is the maximal number of linearly independent columns of this matrix. If you remove (even noise) components from your data, you remove information, but not column or rows (there are still an equal amount of channels and time points present). This makes the data linearly dependent - or: rank deficient.

This can be visualized in MNE-Python easily when plotting the data covariance matrix. Let's compute the data covariance and visualize it - first for the original data, then for the Maxwell-filtered data.
In the singular value plots, you can see a vertical line at the estimated rank of the data. This vertical line coincides with the cliff of the singular value spectrum. In some cases, the rank estimation can fail - you might see another cliff that suggests an even _lower_ rank than the vertical line. The true rank of your data in then the _lower_ number.


<div class="alert alert-block alert-info">
    <b>Rank estimation of covariance matrices</b>
    <br> <br>
      "It is advisable to check the quality of the data covariance matrix by inspecting the rank and condition number, and also the singular value spectrum of the matrix, which is obtained by a Singular Value Decomposition (SVD). Plotting the singular values on a logarithmic axis provides an indication of the effective numerical rank of the covariance matrix. Typically, the singular value spectrum of an ill-conditioned matrix shows a "cliff" at its effective rank, with the numerically irrelevant components having singular values several orders of magnitude smaller than the rest. Sometimes, the spectrum can show two or more such cliffs, e.g., in the case of combined channel types or when the data has been processed with SSS. In such cases, a close inspection of the singular value spectrum is advisable, as conventional rank estimation via SVD can fail." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


In [8]:
# compute the data covariance
data_cov = mne.compute_covariance(epochs_orig, tmin=0.05, tmax=0.15,
                                  method='empirical')

In [ ]:
# visualize the covariance matrix and the singular value spectrum
mne.viz.plot_cov(data_cov, info=epochs_orig.info);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Change the code above to instead plot the Maxwell-filtered data. What do you notice? </li>
    </ul>
</div>

## Beamforming SSS'ed data

Let's attempt to beamform the Maxwell-filtered data. For beamforming, we need the data covariance matrix to construct the beamformer spatial filter. During the computation, the inverse of the covariance matrix is taken - and that inverse is ill-defined if the matrix is rank-deficient. 

<div class="alert alert-block alert-info">
    <b>Rank estimation of covariance matrices</b>
    <br> <br>
      "If the estimate of the data covariance is unreliable, and thus proves to be ill-conditioned, a simple mathematical inverse of this matrix is either impossible, causing the beamformer computation to fail completely, or the used ill-conditioned covariance matrix will lead to poor beamformer results." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


### Prepare beamforming

We first have to load the forward model - here, we load a _volumetric_ forward model. If you want to learn more about how to compute forward models, check out the notebook `Forward_modelling.ipynb`.

We then, using the Maxwell-filtered data, pick the magnetometers only, to not also have to deal with different channel types. We again pick the data of the right visual field presentations - and create the evoked field. We compute the data covariance matrix (again, because we picked channels just now).

In [10]:
# load the precomputed MEG forward model from disk
fname_fwd = data_path / 'sample_audvis-meg-vol-7-fwd.fif'
fwd_meg = mne.read_forward_solution(fname_fwd)

In [12]:
# compute evoked fields
epochs_sss.load_data().pick(picks=['mag'])
evoked_sss = epochs_sss.average()

In [13]:
# compute covariance matrix
data_cov_sss = mne.compute_covariance(epochs_sss, tmin=0.05, tmax=0.15,
                                  method='empirical')

### Compute beamformer and apply to evoked data

Now we can compute the beamformer and apply it to the evoked fields. We choose the Linearly Constrained Minimum Variance beamformer (LCMV), which is expecting time-resolved data, such as an evoked field.

We use the following ingredients to compute the beamformer:
- _data covariance matrix_: This covariance matrix should represent the data.
- _the data_: The beamformer will be applied to this - we pass in our evoked object we have created above.
- _the forward model_: The one we loaded from disk.
- _orientation_: We ask the beamformer to compute the source orientations that maximize power for us. That is done using `pick_ori='max-power'`.
- _regularization_: We choose for no regularization `reg=0.0`, more on that below.


In [14]:
from mne.beamformer import make_lcmv, apply_lcmv

filters = make_lcmv(epochs_sss.info, fwd_meg, data_cov=data_cov_sss, reg=0.0, pick_ori='max-power')
stc_sss = apply_lcmv(evoked=evoked_sss, filters=filters)

We can plot the brain and time course:

In [ ]:
stc_sss.crop(-0.01, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Does this source reconstruction look as expected for visually evoked activity? </li>
      <li> Below you find the code to compute the same source reconstruction on the original data. Compare the outcomes. Can you see the impact of the severe rank deficiency? </li>
    </ul>
</div>

In [ ]:
# beamform the original data and plot
epochs_orig.load_data().pick(picks=['mag'])
evoked_orig = epochs_orig.average()
data_cov_orig = mne.compute_covariance(epochs_orig, tmin=0.05, tmax=0.15,
                                  method='empirical')

filters = make_lcmv(epochs_orig.info, fwd_meg, data_cov=data_cov_orig, reg=0.0, pick_ori='max-power')
stc_orig = apply_lcmv(evoked=evoked_orig, filters=filters)

stc_orig.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

### Rescuing Maxwell-filtered data

Let's see what we can do to "rescue" the Maxwell-filtered data and succeed in source reconstructing the activity.

There are four different strategies we can try:
1. Regularization
2. Truncated pseudo-inverse
3. Spatial whitening
4. Use a different source reconstruction method.

We will look into these options in more detail below. Not every method will work for every data set (except maybe #4). Methods 1-3 can also be combined - you can play around with the parameters yourself and see what impact they have when combined.

#### 1. Tikhonov regularization

We can de-correlate the columns of our rank-deficient covariance matrix, by adding to the diagonal. This process is called Tikhonov regularization. The values to add to the diagonal are expressed as ratios/percentages of the sensor power. In MNE-Python, we use the `reg` parameter for that.

In our example, we will add 10% of the global sensor power: `reg=0.1`. 

The downside of regularization is that it decreases spatial resolution. 

<div class="alert alert-block alert-info">
    <b>Tikhonov regularization</b>
    <br> <br>
      "To ensure a numerically stable inversion of the covariance matrix, one can use a truncated pseudo-inverse, or use “diagonal loading” as a regularization technique (Hillebrand and Barnes, 2003; 2005). Diagonal loading makes the data covariance matrix full-rank by adding a small constant to the diagonal elements of the matrix (Vrba and Robinson, 2000). The regularization of the covariance matrix results in a broader passband of the spatial filter, which increases the output SNR of the filter, but also spatially blurs the source estimates (Brookes et al., 2008). Thus, a trade-off between increasing output SNR and decreasing spatial resolution exists and makes the choice of the regularization parameter 𝜆 crucial for beamformer performance." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


In [ ]:
filters = make_lcmv(epochs_sss.info, fwd_meg, data_cov=data_cov_sss, pick_ori='max-power', reg=0.1)
stc_meg = apply_lcmv(evoked=evoked_sss, filters=filters)

stc_meg.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Does this make the source estimate look better than before? </li>
      <li> Can you justify the claim that regularization decreases spatial resolution? Try increasing the parameter and see what changes. </li>
    </ul>
</div>

#### 1. Truncated pseudo-inverse

A downside of regularization is that you need to guesstimate the regularization parameter - and that usually not to fit one data set, but many. An alternative to Tikhonov regularization is using a truncated pseudo-inverse. 

Here, only the part of the covariance matrix that spans the non-rank-deficient space will be inverted - and then projected back onto the full space to find the pseudo-inverse of the covariance matrix.

For that, we need to pass the effective rank to the beamformer step. We have seen above in the singular value plots that the effective rank of the magnetometers is 72.

In [ ]:
data_rank = dict(mag=72)
filters = make_lcmv(epochs_sss.info, fwd_meg, data_cov=data_cov_sss, pick_ori='max-power',
                    rank=data_rank)
stc_meg = apply_lcmv(evoked=evoked_sss, filters=filters)

stc_meg.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

#### 3. Spatial whitening

Often used with combined channel types (see notebook `Combining_channel_types.ipynb`), whitening is also useful with rank deficient data as it decorrelates the noise components in the data - and thus directly helps with linearly dependent components of the data.

<div class="alert alert-block alert-info">
    <b>Spatial whitening</b>
    <br> <br>
      "Whitening, also called pre-whitening, is a linear operation that intends to decorrelate and scale the noise components in the data. The term whitening refers to the color of the noise being white, i.e., having a covariance matrix that equals the Identity matrix. To achieve this, the procedure uses a so-called noise covariance matrix, a channel-level covariance matrix which can be computed on an empty room recording or a pre-stimulus quiescent baseline." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


In [ ]:
noise_cov_sss = mne.compute_covariance(epochs_sss, tmax=0.0,
                                        method='empirical')
filters = make_lcmv(epochs_sss.info, fwd_meg, data_cov=data_cov_sss,
                    noise_cov=noise_cov_sss, reg=0.0,
                    pick_ori='max-power')
stc_meg = apply_lcmv(evoked=evoked_sss, filters=filters)

stc_meg.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> As mentioned above, these three measures can also be combined. Do you think this is necessary for this data set? </li>
      <li> You can play around and mix the different methods. </li>     
    </ul>
</div>

#### 4. Use a different inverse solution

If all this fails - or to begin with - you can also use a different source reconstruction technique, e.g. Minimum Norm Estimation (MNE). The class of MNEs is not so sensitive rank-deficient data.

For that, we need to load a different forward model - MNEs work on surface estimates. We can just re-use the noise covariance matrix we have computed for whitening, and we don't need a data covariance matrix.

In [22]:
# load the precomputed surface forward model from disk
fname_fwd = data_path / 'sample_audvis-meg-oct-6-fwd.fif'
fwd = mne.read_forward_solution(fname_fwd)

# import MNE methods
from mne.minimum_norm import make_inverse_operator, apply_inverse

# compute and apply the inverse operator
inv = make_inverse_operator(evoked_sss.info, fwd, noise_cov_sss)
stc_mne = apply_inverse(evoked_sss, inv, method='dSPM')

# plot
stc_mne.crop(-0.05, 0.15).plot(hemi='both', subjects_dir=subjects_dir, subject='sample');

## Take home messages

Here are the top things to pay special attention to when using severely rank-deficient data for beamforming:

- Visualize the covariance matrices to get an accurate rank estimate via the singular value spectra.
- Mitigate the rank deficiency by one or a combination of the following methods: Thikonov regularization, truncated pseudo inverse, and spatial whitening.
- If all fails, use a different source reconstruction method.
